In [2]:
# Turing Machine to calculate Fibonacci Sequence
# input = n
# output = F(n)

In [186]:
def parse(defn):
    states = {}
    for s in defn:
        state_name = s.split(":")[0]
        t_string = s.split(":")[1].strip().split(" ")
#         print(t_string)
#         assert (len(t_string) == 3)
        transition = {}
        for t in t_string:
            read = t[0]
            write = t[1]
            direction = t[2]
            next_state = t[3:]
            transition[read] = {"write": write, "direction": direction, "next":next_state}
        states[state_name] = transition
    return states

def run(machine, tape, debug=False):
    state = "START"
    head = 0
    head_char = None
    write = None
    while state != "HALT":
        head_char = tape[head]
        write = machine[state][head_char]["write"]
        tape[head] = write
        direction = machine[state][head_char]["direction"]
        next_state = machine[state][head_char]["next"]
        
        if debug == True:
            print_tape(tape, head)
            print("state ", state, "read", head_char, "write" , write)
            
        if direction == "R":
            head += 1
            if head >= len(tape):
                tape.append("_")
        else:
            head -= 1
            if head < 0:
                tape.insert(0,"_")
                head += 1
                
        state = next_state
        
    if debug == True:
        print_tape(tape, head)
        print("state ", state)
        
    return tape

def print_tape(tape, head):
    for i in range(0,head):
        print(tape[i], end="")
    print("^", end="")
    for i in range(head, len(tape)):
        print(tape[i], end="")
    print()
    

In [181]:
#Format: State number: (input<0,1,_>,output<0,1,_>,head<L,R>,newstate) x 3
#Machine: reads input, writes output to tape, moves in the direction specified
#Go to the end of the input
machine = [
    "START: 00RSTART 11RSTART __Rinit0",
    #write 00_01_ (we are using 00 to represent number 0 and 01 to represent 1 b/c we need to mark chars as read or not)
    "init0: 00Rinit1 10Rinit1 _0Rinit1",
    "init1: 00Rinit2 10Rinit2 _0Rinit2",
    "init2: 0_Rinit3 1_Rinit3 __Rinit3",
    "init3: 00Rinit4 10Rinit4 _0Rinit4",
    "init4: 01Rinit5 11Rinit5 _1Rinit5",
    "init5: 0_Lback0 1_Lback0 __Lback0", #moving L because we wrote a 1 already, so now we want to go back
    # tape now is "n_0*_1", go back to n
    "back0: 00Lback0 11Lback0 __Lback1",
    "back1: 00Lback1 11Lback1 __Lback2",
    "back2: 00Lback2 11Lback2 __Rcheckn0", #moving R because we were at a blank (to the left of n)
    #back at beginning, check to see if n == 0
    "checkn0: 00Rcheckn0 11Rsub0 __Rclean",
    # state 7 means n != 0
    # move to the right of n
    "sub0: 00Rsub0 11Rsub0 __Lsub1",
    # now at least significant bit of n, subtract 1
    "sub1: 01Lsub1 10Lsub2 __RHALT", # should never reach _
    "sub2: 00Lsub2 11Lsub2 __Rfib0",
    # state clean means n == 0  
    "clean: 00LHALT 11LHALT __LHALT",
    # state fib0 means we have to start adding the stuff (we are at leftmost bit of n)
    "fib0: 00Rfib0 11Rfib0 __Rfib1",
    "fib1: 00Rfib1 11Rfib1 __Rfib2", # at leftmost bit of the first number
    "fib2: 00Rfib2 11Rfib2 __Rfib3",
    "fib3: 00Rfib3 11Rfib3 _1Lfib4", # for now just keep on adding 1's to the end of the list (eventually do the sum)
    "fib4: 00Lfib4 11Lfib4 __Lback0", # we need to go back one extra "_" before going to the back subroutine
]

In [182]:
mach = parse(machine)
run(mach, tape = list("1011"), debug=True)

^1011
state  START read 1 write 1
1^011
state  START read 0 write 0
10^11
state  START read 1 write 1
101^1
state  START read 1 write 1
1011^_
state  START read _ write _
1011_^0
state  init0 read _ write 0
1011_0^0
state  init1 read _ write 0
1011_00^_
state  init2 read _ write _
1011_00_^0
state  init3 read _ write 0
1011_00_0^1
state  init4 read _ write 1
1011_00_01^_
state  init5 read _ write _
1011_00_0^1_
state  back0 read 1 write 1
1011_00_^01_
state  back0 read 0 write 0
1011_00^_01_
state  back0 read _ write _
1011_0^0_01_
state  back1 read 0 write 0
1011_^00_01_
state  back1 read 0 write 0
1011^_00_01_
state  back1 read _ write _
101^1_00_01_
state  back2 read 1 write 1
10^11_00_01_
state  back2 read 1 write 1
1^011_00_01_
state  back2 read 0 write 0
^1011_00_01_
state  back2 read 1 write 1
^_1011_00_01_
state  back2 read _ write _
_^1011_00_01_
state  checkn0 read 1 write 1
_1^011_00_01_
state  sub0 read 0 write 0
_10^11_00_01_
state  sub0 read 1 write 1
_101^1_00_01_
state 

state  fib4 read 1 write 1
_0100_00_01^_1111111
state  fib4 read _ write _
_0100_00_0^1_1111111
state  back0 read 1 write 1
_0100_00_^01_1111111
state  back0 read 0 write 0
_0100_00^_01_1111111
state  back0 read _ write _
_0100_0^0_01_1111111
state  back1 read 0 write 0
_0100_^00_01_1111111
state  back1 read 0 write 0
_0100^_00_01_1111111
state  back1 read _ write _
_010^0_00_01_1111111
state  back2 read 0 write 0
_01^00_00_01_1111111
state  back2 read 0 write 0
_0^100_00_01_1111111
state  back2 read 1 write 1
_^0100_00_01_1111111
state  back2 read 0 write 0
^_0100_00_01_1111111
state  back2 read _ write _
_^0100_00_01_1111111
state  checkn0 read 0 write 0
_0^100_00_01_1111111
state  checkn0 read 1 write 1
_01^00_00_01_1111111
state  sub0 read 0 write 0
_010^0_00_01_1111111
state  sub0 read 0 write 0
_0100^_00_01_1111111
state  sub0 read _ write _
_010^1_00_01_1111111
state  sub1 read 0 write 1
_01^11_00_01_1111111
state  sub1 read 0 write 1
_0^011_00_01_1111111
state  sub1 read 1 writ

In [183]:
len("11111111111")

11

In [184]:
add = [
    "START: 01Raddnc1 11Raddnc2 __L0nc",
    "addnc1: 00R0nc 11R1nc",
    "addnc2: 00RSTART 11RSTART",
    
    "addc0: 01Raddc1 11Raddc2 __L1nc",
    "addc1: 00R1nc 11R0c",
    "addc2: 00Raddc0 11Raddc0",
    
    "0nc: 00R0nc 11R0nc __R0nc1",
#     "0nc1: 01R0nc2 11R0nc3 __Lwrite0nc",
    "0nc1: 01R0nc2 11R0nc3 __Rfinish",
    "0nc2: 00Rwrite0nc 11Rwrite1nc",
    "0nc3: 00R0nc1 11R0nc1",
    
    "1nc: 00R1nc 11R1nc __R1nc1",
    "1nc1: 01R1nc2 11R1nc3 __Lwrite1nc",
#     "1nc1: 01R1nc2 11R1nc3 __RHALT",
    "1nc2: 00Rwrite1nc 11Rwrite0c",
    "1nc3: 00R1nc1 11R1nc1",
    
    "0c: 00R0c 11R0c __R0c1",
#     "0c1: 01R0c2 11R0c3 __Lwrite0c",
    "0c1: 01R0c2 11R0c3 __Rfinish",
    "0c2: 00Rwrite0c 11Rwrite1c",
    "0c3: 00R0c1 11R0c1",
    
    "write0nc: 00Rwrite0nc 11Rwrite0nc __Rwrite0nc1",
    "write0nc1: 00Rwrite0nc1 11Rwrite0nc1 _0Rwrite0nc2",
    "write0nc2: 00Rwrite0nc1 11Rwrite0nc1 _0Lgobacknc",
#     "write0nc1: _0Rwrite0nc2",
#     "write0nc2: _0Lgobacknc",
    
    "write1nc: 00Rwrite1nc 11Rwrite1nc __Rwrite1nc1",
    "write1nc1: 00Rwrite1nc1 11Rwrite1nc1 _0Rwrite1nc2",
    "write1nc2: 00Rwrite1nc1 11Rwrite1nc1 _1Lgobacknc",
#     "write1nc1: _0Rwrite1nc2",
#     "write1nc2: _1Lgobacknc",
    
    "write0c: 00Rwrite0c 11Rwrite0c __Rwrite0c1",
    "write0c1: 00Rwrite0c1 11Rwrite0c1 _0Rwrite0c2",
    "write0c2: 00Rwrite0c1 11Rwrite0c1 _0Lgobackc",
#     "write0c1: _0Rwrite0c2",
#     "write0c2: _0Lgobackc",
    
    "write1c: 00Rwrite1c 11Rwrite1c __Rwrite1c1",
    "write1c1: 00Rwrite1c1 11Rwrite1c1 _0Rwrite0c2",
    "write1c2: 00Rwrite1c1 11Rwrite1c1 _1Lgobackc",
#     "write1c1: _0Rwrite0c2",
#     "write1c2: _1Lgobackc",
    
    "gobacknc: 00Lgobacknc 11Lgobacknc __Lgobacknc1",
    "gobacknc1: 00Lgobacknc1 11Lgobacknc1 __Lgobacknc2",
    "gobacknc2: 00Lgobacknc2 11Lgobacknc2 __RSTART",
#     "gobacknc3: 00Lgobacknc3 11Lgobacknc3 __LSTART",
    
    "gobackc: 00Lgobackc 11Lgobackc __Lgobackc1",
    "gobackc1: 00Lgobackc1 11Lgobackc1 __Lgobackc2",
    "gobackc2: 00Lgobackc2 11Lgobackc2 __Raddc0",
#     "gobackc3: 00Lgobackc3 11Lgobackc3 __Laddc0",
    "finish: 00Lfinish 11Lfinish __Lfinish1",
    "finish1: 00Lfinish1 11Lfinish1 __Lfinish2",
    "finish2: 00Lfinish2 11Lfinish2 __RHALT",
]

add_mach = parse(add)

In [185]:
run(add_mach, list("0001_010101"), debug=True)

^1001_010101
state  START read 0 write 1
1^001_010101
state  addnc1 read 0 write 0
10^01_010101
state  0nc read 0 write 0
100^1_010101
state  0nc read 1 write 1
1001^_010101
state  0nc read _ write _
1001_^110101
state  0nc1 read 0 write 1
1001_1^10101
state  0nc2 read 1 write 1
1001_11^0101
state  write1nc read 0 write 0
1001_110^101
state  write1nc read 1 write 1
1001_1101^01
state  write1nc read 0 write 0
1001_11010^1
state  write1nc read 1 write 1
1001_110101^_
state  write1nc read _ write _
1001_110101_^0
state  write1nc1 read _ write 0
1001_110101_0^1
state  write1nc2 read _ write 1
1001_110101_^01
state  gobacknc read 0 write 0
1001_110101^_01
state  gobacknc read _ write _
1001_11010^1_01
state  gobacknc1 read 1 write 1
1001_1101^01_01
state  gobacknc1 read 0 write 0
1001_110^101_01
state  gobacknc1 read 1 write 1
1001_11^0101_01
state  gobacknc1 read 0 write 0
1001_1^10101_01
state  gobacknc1 read 1 write 1
1001_^110101_01
state  gobacknc1 read 1 write 1
1001^_110101_01
state 

In [211]:
#testing add
for i in range(10):
    for j in range(i, 10):
        t = ("0".join(bin(j)[2:]) + "0_" + "0".join(bin(i)[2:]) + "0")[::-1]
        out = run(parse(add), list(t), debug=False)
        num = int(("".join(out)).split("_")[-1][1::2][::-1], 2)
        if i+j != num:
            print(t)
            print("".join(out))
            print(i, j, num)
        

0101_0101
_1111_1111_000001
3 3 4
0101_010101
_1111_111111_00000001
3 7 8
010001_010101
_111011_111111_00000001
5 7 8
000101_000101
_101111_101111_00000001
6 6 8
000101_010101
_101111_111111_01000001
6 7 9
010101_010101
_111111_111111_00000001
7 7 8


In [196]:
t = ("0".join(bin(6)[2:]) + "0_" + "0".join(bin(2)[2:]) + "0")[::-1]
print(t)

0001_000101


In [199]:
"01000001"[1::2]

'1001'

In [200]:
int("1001", 2)

9